# Model selection

In this notebook we will do the model selection and predictions.

# TOC

* [1 Loading the data](#1-Loading-the-data)
* [2 Mean prediction](#2-Mean-prediction)
* [3 Make loss function](#3-Make-loss-function)
* [4 Validation generation](#4-Validation-generation)
* [5 Hyperparameter optimization](#5-Hyperparameter-optimization)
    * [5.1 Linear Regression](#5.1-Linear-Regression)
    * [5.2 Lasso](#5.2-Lasso)
    * [5.3 Ridge](#5.3-Ridge)
    * [5.4 KNN](#5.4-KNN)
    * [5.5 Extremely Randomized Trees](#5.5-Extremely-Randomized-Trees)
    * [5.6 Gradient Boosting Decision Tree](#5.6-Gradient-Boosting-Decision-Tree)
    * [5.7 Neural Network](#5.7-Neural-Network)
* [6 Ensembling](#6-Ensembling)

# 1 Loading the data

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn import model_selection
from sklearn import metrics

In [ ]:
# Set the random seeds
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

# Possible more fixes for non-determinism
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926
# https://github.com/keras-team/keras/issues/2280#issuecomment-366542480
import os
from keras import backend as k

os.environ['PYTHONHASHSEED'] = '0'
sess = tf.Session(graph=tf.get_default_graph())

# Limit operation to 1 thread for deterministic results.
# NOTE: This will slow down the operation
# session_conf = tf.ConfigProto(
#     intra_op_parallelism_threads=1,
#     inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)

k.set_session(sess)

In [ ]:
generated_data = Path('.').absolute().joinpath('generated_data')

target_name = 'item_cnt_month'
dt_data = pd.read_hdf(generated_data.joinpath('dt_data.hdf'), key='dt_data')
non_dt_data = pd.read_hdf(generated_data.joinpath('non_dt_data.hdf'), key='non_dt_data')

In [ ]:
train_dt = dt_data.loc[dt_data.loc[:, 'date_block_num'] < dt_data.loc[:, 'date_block_num'].max()].copy()
target = train_dt.loc[:, [target_name]]
train_dt.drop(target_name, axis=1, inplace=True)
test_dt = dt_data.loc[dt_data.loc[:, 'date_block_num'] == dt_data.loc[:, 'date_block_num'].max()].copy()
test_dt.drop(target_name, axis=1, inplace=True)

In [ ]:
train_dt.head()

In [ ]:
target.head()

In [ ]:
test_dt.head()

In [ ]:
# NOTE: We have normalized 'date_block_num' in non_dt_data, so we should be careful to use this as a filter
#       as this is now a float value
#       Instead we use the index values
train_non_dt = non_dt_data.iloc[:train_dt.shape[0]].copy()
train_non_dt.drop(target_name, axis=1, inplace=True)
test_non_dt = non_dt_data.iloc[train_dt.shape[0]:].copy()
test_non_dt.drop(target_name, axis=1, inplace=True)

In [ ]:
train_non_dt.head()

In [ ]:
test_non_dt.head()

In [ ]:
del dt_data
del non_dt_data
gc.collect()

# 2 Mean prediction

Before we start we should do some rudimental model predictions.

The file `sample_submission.csv.gz` contains the constant prediction `0.5`, and gives the score `1.23646` against the kaggle site.

Furthermore we know that the optimal prediction for a constant is a target mean (of the ground truth).
Nevertheless, we can probe the leaderboard with target mean of the training set.

In [ ]:
mean_prediction = test_dt.loc[:, ['ID']]
mean_prediction.loc[:, 'item_cnt_month'] = target.loc[:, 'item_cnt_month'].mean()

# Set ID as index
mean_prediction.set_index('ID', inplace=True)

mean_prediction.to_csv(generated_data.joinpath('mean_prediction.csv'))
mean_prediction.head()

The mean prediction gave a score of `1.21744`, which is worse than our initial submission.
This means that on average, predictions with lower values are preferred over predicitions with higher values.

We can in fact use this to probe the leaderboard. As we know that the constant target mean of the ground thruth gives the lowest score, we can check whether `0.2` is a minimum (at least of the public test set).

In [ ]:
prediction_02 = mean_prediction.copy()
prediction_02.loc[:, 'item_cnt_month'] = 0.2
prediction_02.to_csv(generated_data.joinpath('prediction_02.csv'))
prediction_02.head()

This degraded the score to `1.22032`.

We could continue to probe the leader board like this to find the minimum to get a direction on what our prediction mean should be close to. However, we must bear in mind that we are only probing the public part of the test set, so we must use this technique with care.

# 3 Make loss function

It appears that the RSME is not available as a loss function out of the box, so we define it ourselves.
This also give us the oppurtunity to clip the predictions to $[0,20]$

In [ ]:
def rmse_clip(ground_truth, predictions):
    """
    Returns the root mean squared error of the predictions
    
    The root mean squared error is defined by:
    $\sqrt {\frac {\sum _{t=1}^{T}({\hat {y}}_{t}-y_{t})^{2}}{T}}$
    
    Note
    ----
    This version clips the predictions to [0, 20]
    
    Parameters
    ----------
    ground_truth : array, shape (n_samples,)
        The correct prediction
    prediction : array, shape (n_samples,)
        The predictions
        
    Returns
    -------
    rmse : float
        The root mean squared error
    """
    return np.sqrt(metrics.mean_squared_error(ground_truth, predictions.clip(0, 20)))

# 4 Validation generation

As we want to predict for the next month, we know that train-test is split by time (we would like to predict for month $34$).

In addition, from [0_EDA_raw_data.ipynb](0_EDA_raw_data.ipynb), we saw that different band of item ids were removed (i.e. non-random row numbers were removed in the training set). 

As a rule of thumb we should mimic the validation in the similar manner. The time component is fairly straigth forward. The question is whether it makes sense to take out bands of item id in addition. This is of course testable, and due to time constraints we will just split by time here.

**NOTE**: We could have used `sklearn.model_selectionTimeSeriesSplit(n_splits=n)` to split the dataset. However, as we have quite big training set and since training of the models takes time, we will use only one split. Furthermore, we will do a simple parameter tuning where one parameter is tuned at the time. Thus, we make the split and the tuning ourselves instead of using `sklearn.model_selection.GridSearchCV` (it turns out that non-native `sklearn`-models needs some pathcing to work anyway). 

In [ ]:
n_train = train_dt.shape[0]
n_test = test_dt.shape[0]

In [ ]:
n_test/n_train

We see that the test-set is quite small in comparison to the train set. 

There are several ways to make a proper split for the validation set. We could for example take into account what `item_id` and `shop_id` which are present, where we are in a seasonal trend and so forth. In order to simplify the process, we just want to take care that there are enough samples in the validation set. Therefore, we make a $70-30$ split of the dataset.

In [ ]:
pct_train = 0.7
train_split = int(n_train*pct_train)
indices_from_original_train = np.array(range(n_train))
train_indices = indices_from_original_train[:train_split]
val_indices = indices_from_original_train[train_split:]

In [ ]:
x_train_unscaled = train_dt.iloc[train_indices]
x_val_unscaled = train_dt.iloc[val_indices]

x_train_scaled = train_non_dt.iloc[train_indices]
x_val_scaled = train_non_dt.iloc[val_indices]

y_train = target.iloc[train_indices]
y_val = target.iloc[val_indices]

# 5 Hyperparameter optimization

In order not to run the same fittings several times we make a dictionary to take care of the book-keeping.

In [ ]:
model_dir = generated_data.joinpath('models')
model_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
hyper_df_path = generated_data.joinpath('hyper_df.hdf')
if hyper_df_path.is_file():
    hyper_df = pd.read_hdf(hyper_df_path, key='hyper_df')
else:
    hyper_df = pd.DataFrame(columns=['name', 'train_score', 'val_score'])

In [ ]:
def pickle_object(save_dir, file_name, obj):
    """
    Pickles a model
    
    Parameters
    ----------
    save_dir : Path
        Path to save to
    file_name : str
        The name of the file
    obj : object
        The object to store
    """
    
    file_path = save_dir.joinpath(f'{file_name}.pkl')
    with file_path.open('wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        print(f'Saved to {file_path}')

In [ ]:
def unpickle_object(save_dir, file_name):
    """
    Pickles a model
    
    Parameters
    ----------
    model_dir : Path
        Path to the models
    model_name : str
        The name of the model
        
    Returns
    -------
    model : model object
        The model to load
    """
    
    file_path = Path(save_dir).joinpath(f'{file_name}.pkl')
    with file_path.open('rb') as f:
        obj = pickle.load(f)
        print(f'Loaded from {file_path}')
    return obj

In [ ]:
def add_to_hyper_df(hyper_df, hyper_df_path, model_name, train_score, val_score):
    """
    Add the results of a model to the hyper parameter data frame
    
    Parameters
    ----------
    hyper_df : DataFrame
        The data frame to append
    hyper_df_path : Path
        Path to the stored data frame
    model_name : str
        Name of the model
    train_score : float
        The training score
    val_score : float
        The validation score
        
    Returns
    -------
    hyper_df : DataFrame
        The appended data frame
    """
    
    hyper_df = hyper_df.append({'name': model_name, 
                                'train_score': train_score,
                                'val_score': val_score},
                               ignore_index=True)
    hyper_df.to_hdf(hyper_df_path, key='hyper_df')
    
    return hyper_df

In [ ]:
def best_model(model_base_name, hyper_df, val_score):
    """
    Check if the model has the best score among the model base
    
    Parameters
    ----------
    model_base_name : str
        Name of the model base
    hyper_df : DataFrame
        The dataframe containing all the scores
    val_score : float
        The validation score
        
    Returns
    -------
    result : bool
        Whether the validation score is the best 
    """
    
    rows = [row for row in hyper_df.loc[:, 'name'] if model_base_name in row]
    
    if len(rows) == 0:
        return True
    
    best_score = sorted(hyper_df.loc[hyper_df.loc[:, 'name'].isin(rows), 'val_score'].values)[0]
    
    if val_score < best_score:
        return True
    else:
        return False

In [ ]:
def plot_train_val(hyper_df, model_base_name, param, exclude=[]):
    """
    Plot the train-validation curve
    
    Parameters
    ----------
    model_base_name : str
        Name of the model base
    hyper_df : DataFrame
        The dataframe containing all the scores
    param : str
        Parameter to check
    exclude : list
        Parameter to exclude
    """
   
    rows = [row for row in hyper_df.loc[:, 'name'] 
            if model_base_name in row 
            and param in row]
    
    if len(exclude) != 0:
        for excl in exclude:
            rows = [row for row in rows if excl not in row]
            
    train_scores = hyper_df.loc[hyper_df.loc[:, 'name'].isin(rows), 'train_score'].values
    val_scores = hyper_df.loc[hyper_df.loc[:, 'name'].isin(rows), 'val_score'].values
    parameter_vals = [float(val.split(f'_{param}_')[-1].split('_')[0]) for val in rows]
    
    fig, ax = plt.subplots()
        
    ax.plot(parameter_vals, train_scores, label='Train')
    ax.plot(parameter_vals, val_scores, label='Validation')
    ax.set_xlabel(param)
    ax.set_ylabel('Error')
    ax.grid(True)
    ax.legend(loc='best', fancybox=True, framealpha=0.5)

## 5.1 Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

There are no real hyperparameters to tune in linear regression (other than choosing wheter we should include the intersect or not).

In [ ]:
model_name = 'lin_reg'

if not model_name in hyper_df.loc[:, 'name'].values:
    lin_reg = LinearRegression()
    lin_reg.fit(x_train_scaled, y_train)
    lin_reg_train_score = rmse_clip(y_train, lin_reg.predict(x_train_scaled))
    lin_reg_val_score = rmse_clip(y_val, lin_reg.predict(x_val_scaled))
    pickle_object(model_dir, model_name, lin_reg)
    hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, lin_reg_train_score, lin_reg_val_score)
else:
    lin_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
    lin_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
    lin_reg = unpickle_object(model_dir, model_name)

In [ ]:
print(f'Train score: {lin_reg_train_score:.3f}')
print(f'Validation score: {lin_reg_val_score:.3f}')

## 5.2 Lasso

In [ ]:
from sklearn.linear_model import Lasso

We tune the strength of the `L1` regularizer through the `alpha` parameter

In [ ]:
model_base_name = 'lasso_reg'
model_name_template = f'{model_base_name}_alpha_{{}}'

for alpha in [1, 2, 3, 4, 8]:
    print(f'Processing alpha={alpha}')
    model_name = model_name_template.format(alpha)
    if not model_name in hyper_df.loc[:, 'name'].values:
        lasso_reg = Lasso(alpha=alpha)
        # NOTE: We ravel in order to avoid warnings
        lasso_reg.fit(x_train_unscaled, y_train.values.ravel())
        lasso_reg_train_score = rmse_clip(y_train, lasso_reg.predict(x_train_unscaled))
        lasso_reg_val_score = rmse_clip(y_val, lasso_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, lasso_reg_val_score):
            pickle_object(model_dir, model_base_name, lasso_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, lasso_reg_train_score, lasso_reg_val_score)
    else:
        lasso_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        lasso_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        lasso_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'alpha')

## 5.3 Ridge

In [ ]:
from sklearn.linear_model import Ridge

We tune the strength of the `L2` regularizer through the `alpha` parameter

In [ ]:
model_base_name = 'ridge_reg'
model_name_template = f'{model_base_name}_alpha_{{}}'

for alpha in [1, 2, 3, 4, 8]:
    print(f'Processing alpha={alpha}')
    model_name = model_name_template.format(alpha)
    if not model_name in hyper_df.loc[:, 'name'].values:
        ridge_reg = Ridge(alpha=alpha)
        # NOTE: We ravel in order to avoid warnings
        ridge_reg.fit(x_train_unscaled, y_train.values.ravel())
        ridge_reg_train_score = rmse_clip(y_train, ridge_reg.predict(x_train_unscaled))
        ridge_reg_val_score = rmse_clip(y_val, ridge_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, ridge_reg_val_score):
            pickle_object(model_dir, model_base_name, ridge_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, ridge_reg_train_score, ridge_reg_val_score)
    else:
        ridge_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        ridge_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        ridge_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'alpha')

The ridge classifier appers to be in a bad health at the moment (given the ill-condition warnings). This is further discussed [here](https://github.com/scikit-learn/scikit-learn/issues/10517). Therefore, we will drop using the ridge classifier.

## 5.4 KNN

**NOTE**: The KNN uses forever to run, and it appears that it only uses one processor (even for `njobs=-1`). Note that KNN scales rather badly with number of points as seen [here](https://www.thekerneltrip.com/machine/learning/computational-complexity-learning-algorithms/) and [here](https://cs.stanford.edu/people/ang/papers/nips06-mapreducemulticore.pdf). To overcome the problem, we could use PCA analysis or t-SNE for dimensionality reduction. Similar arguments also holds for SVM as indicated [here](https://stackoverflow.com/questions/18165213/how-much-time-does-take-train-svm-classifier)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
run_knn = False

In [ ]:
model_name = 'knn_reg_n_neighbors_1'

if run_knn:
    if not model_name in hyper_df.loc[:, 'name'].values:
        knn_reg = KNeighborsRegressor(n_neighbors = 1)
        knn_reg.fit(x_train_scaled, y_train)
        knn_reg_train_score = rmse_clip(y_train, knn_reg.predict(x_train_scaled))
        knn_reg_val_score = rmse_clip(y_val, knn_reg.predict(x_val_scaled))
        pickle_object(model_dir, model_name, knn_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, knn_reg_train_score, knn_reg_val_score)
    else:
        knn_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        knn_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        knn_reg = unpickle_object(model_dir, model_name)

## 5.5 Extremely Randomized Trees

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

We would like to benchmark our results with [this notebook](https://www.kaggle.com/the1owl/playing-in-the-sandbox/notebook), which achieves a RMSE around $0.27$

In [ ]:
model_base_name = 'et_reg'
model_name = f'{model_base_name}_n_estimators_25_max_depth_15'

if not model_name in hyper_df.loc[:, 'name'].values:
    et_reg = ExtraTreesRegressor(n_estimators=25,
                                 n_jobs=-1,
                                 max_depth=15,
                                 random_state=18)
    # NOTE: We ravel in order to avoid warnings
    et_reg.fit(x_train_unscaled, y_train.values.ravel())
    et_reg_train_score = rmse_clip(y_train, et_reg.predict(x_train_unscaled))
    et_reg_val_score = rmse_clip(y_val, et_reg.predict(x_val_unscaled))
    pickle_object(model_dir, model_base_name, et_reg)
    hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, et_reg_train_score, et_reg_val_score)
else:
    et_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
    et_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
    et_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
print(f'Train score: {et_reg_train_score:.3f}')
print(f'Validation score: {et_reg_val_score:.3f}')

Let's see if we can increase the score by first tuning the number of estimators

In [ ]:
model_base_name = 'et_reg'
model_name_template = f'{model_base_name}_n_estimators_{{}}'

# Sklearn documentation:
# https://etboost.readthedocs.io/en/latest/python/python_api.html#module-etboost.sklearn
for n_estimators in [12, 25, 50, 75]:
    print(f'Processing n_estimators={n_estimators}')
    model_name = model_name_template.format(n_estimators)
    if not model_name in hyper_df.loc[:, 'name'].values:
        et_reg = ExtraTreesRegressor(n_estimators=n_estimators,
                                     n_jobs=-1,
                                     max_depth=15,
                                     random_state=18)
        # NOTE: We ravel in order to avoid warnings
        et_reg.fit(x_train_unscaled, y_train.values.ravel())
        et_reg_train_score = rmse_clip(y_train, et_reg.predict(x_train_unscaled))
        et_reg_val_score = rmse_clip(y_val, et_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, et_reg_val_score):
            pickle_object(model_dir, model_base_name, et_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, et_reg_train_score, et_reg_val_score)
    else:
        et_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        et_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        et_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'n_estimators')

We gained a bit to have $75$ estimators, but it also costed time. Let's finally tune the `max_depth`

In [ ]:
model_base_name = 'et_reg'
model_name_template = f'{model_base_name}_max_depth_{{}}'

# Sklearn documentation:
# https://etboost.readthedocs.io/en/latest/python/python_api.html#module-etboost.sklearn
for max_depth in [6, 8, 15, 20]:
    print(f'Processing max_depth={max_depth}')
    model_name = model_name_template.format(max_depth)
    if not model_name in hyper_df.loc[:, 'name'].values:
        et_reg = ExtraTreesRegressor(n_estimators=75,
                                     max_depth=max_depth,
                                     n_jobs=-1,
                                     random_state=18)
        # NOTE: We ravel in order to avoid warnings
        et_reg.fit(x_train_unscaled, y_train.values.ravel())
        et_reg_train_score = rmse_clip(y_train, et_reg.predict(x_train_unscaled))
        et_reg_val_score = rmse_clip(y_val, et_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, et_reg_val_score):
            pickle_object(model_dir, model_base_name, et_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, et_reg_train_score, et_reg_val_score)
    else:
        et_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        et_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        et_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'max_depth', ['n_estimators'])

## 5.6 Gradient Boosting Decision Tree

In [ ]:
from xgboost import XGBRegressor

As with the other estimators, the `xgboost` estimator has several knobs to turn which can be used to find the optimal estimator.

To start with, we have:

Better fitting (increase for reducing underfit)
* max_depth
* subsample
* colsample_bytree
* colsample_bylevel
* eta 
* num_round

Impeeds fitting (increase for reducing overfitting)
* min_child_weight
* lambda
* alpha

We will start with the `max_depth` parameter to investigate the performance

In [ ]:
model_base_name = 'xg_reg'
model_name_template = f'{model_base_name}_max_depth_{{}}'

# Sklearn documentation:
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
for max_depth in [0, 1, 2, 3, 4, 5, 6, 8, 10, 14]:
    print(f'Processing max_depth={max_depth}')
    model_name = model_name_template.format(max_depth)
    if not model_name in hyper_df.loc[:, 'name'].values:
        xg_reg = XGBRegressor(max_depth=max_depth,
                              n_jobs=-1,
                              seed=seed)
        # NOTE: We ravel in order to avoid warnings
        xg_reg.fit(x_train_unscaled, y_train.values.ravel())
        xg_reg_train_score = rmse_clip(y_train, xg_reg.predict(x_train_unscaled))
        xg_reg_val_score = rmse_clip(y_val, xg_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, xg_reg_val_score):
            pickle_object(model_dir, model_base_name, xg_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, xg_reg_train_score, xg_reg_val_score)
    else:
        xg_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        xg_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        xg_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'max_depth')

Let's try to make a submission with this, to see if we are doing any better than our mean prediction

In [ ]:
xg_prediction = test_dt.loc[:, ['ID']]
xg_prediction.loc[:, 'item_cnt_month'] = xg_reg.predict(test_non_dt).clip(0, 20)

# Set ID as index
xg_prediction.set_index('ID', inplace=True)

xg_prediction.to_csv(generated_data.joinpath('xg_prediction.csv'))
xg_prediction.loc[:, 'item_cnt_month'].describe()

This gave a leader board score of $1.07932$, which is better than the mean predictions. Let us improve the validation score even further. From the graph above, it appears that we are [overfitting](http://scikit-learn.org/stable/modules/learning_curve.html#validation-curve) as the training error improves, whereas the validation error is flatting out. Let's increase `min_child_weight`

In [ ]:
model_base_name = 'xg_reg'
model_name_template = f'{model_base_name}_max_depth_14_min_child_weight_{{}}'

# Sklearn documentation:
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
for min_child_weight in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
    print(f'Processing min_child_weight={min_child_weight}')
    model_name = model_name_template.format(min_child_weight)
    if not model_name in hyper_df.loc[:, 'name'].values:
        xg_reg = XGBRegressor(max_depth=14,
                              min_child_weight=min_child_weight,
                              n_jobs=-1,
                              seed=seed)
        # NOTE: We ravel in order to avoid warnings
        xg_reg.fit(x_train_unscaled, y_train.values.ravel())
        xg_reg_train_score = rmse_clip(y_train, xg_reg.predict(x_train_unscaled))
        xg_reg_val_score = rmse_clip(y_val, xg_reg.predict(x_val_unscaled))
        if best_model(model_base_name, hyper_df, xg_reg_val_score):
            pickle_object(model_dir, model_base_name, xg_reg)
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, xg_reg_train_score, xg_reg_val_score)
    else:
        xg_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        xg_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]
        xg_reg = unpickle_object(model_dir, model_base_name)

In [ ]:
plot_train_val(hyper_df, model_base_name, 'min_child_weight')

We observe a minimum around $128$. We could continue to tune the hyperparameters, but we stop here due to time constraints.

## 5.7 Neural network

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

**NOTE**: We also make a costum RMSE for keras

In [ ]:
def rmse_keras_clip(y_true, y_pred):
    """
    Returns the root mean squared error of the predictions
    
    The root mean squared error is defined by:
    $\sqrt {\frac {\sum _{t=1}^{T}({\hat {y}}_{t}-y_{t})^{2}}{T}}$
    
    Note
    ----
    This version clips the predictions to [0, 20]
    
    Parameters
    ----------
    y_true : array, shape (n_samples,)
        The correct prediction (the ground truth)
    y_pred : array, shape (n_samples,)
        The predictions
        
    Returns
    -------
    rmse : float
        The root mean squared error
    """
    return K.sqrt(K.mean(K.square(tf.clip_by_value(y_pred, 0, 20) - y_true), axis=-1))

**NOTE**: RNNs would probably be the best fit for this task, we will for simplicity use plain old multilayer perceptrons

In [ ]:
def build_mlp(input_dim, optimizer, rmse_keras_clip, hidden_layers=1, nodes=32, dropout=0):
    """
    Returns a keras model
    
    Parameters
    ----------
    input_dim : int
        The input dimension
    hidden_layers : int
        The number of hidden layers
    optimizer : str
        The optimizer to use
    rmse_keras_clip : function
        Function which gives the rmse loss function
    nodes : int or array-like, shape (hidden_layers)
        Nodes for all the layers.
        If array-like, each element corresponds to the nodes in the hidden layer
        If int, all hidden layers will have the same number of nodes
    dropout : float or array-like, shape (hidden_layers)
        Dropout for all the layers.
        If array-like, each element corresponds to the dropout values after each hidden layer
        If int, all hidden layers will have the same dropout value
    """
    
    if type(nodes) == int:
        nodes = [nodes] * hidden_layers
    if type(dropout) == float or type(dropout) == int:
        dropout = [dropout] * hidden_layers
    
    model = Sequential()

    model.add(Dense(nodes[0], input_dim=input_dim))
    
    if len(nodes) > 1:
        model.add(Activation('relu'))
    
    for node, drop in zip(nodes[1:], dropout[:-1]):
        model.add(Dropout(drop))
        model.add(Dense(node))
        
        if node != nodes[-1]:
            model.add(Activation('relu'))

    # Add the final layer
    model.add(Dropout(dropout[-1]))
    model.add(Dense(1))
    # NOTE: We use identity as we are dealing with a regression problem
    model.add(Activation('linear'))

    model.compile(loss=rmse_keras_clip,
                  optimizer=optimizer,
                  metrics=[rmse_keras_clip])
    
    model.summary()

    return model

For the neural networks, it makes sense to investigate

Better fitting (increase for reducing underfit)
* Number of neurons per layer
* Number of layers
* Adam/Adadelta/Adagrad/... (observed to lead to more overfitting)
* Batch size

Impeeds fitting (increase for reducing overfitting)
* L2/L1 for weights
* Dropout/Dropconnect
* Static dropconnect

We start by optimizing one layer

In [ ]:
stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', baseline=None)

In [ ]:
model_base_name = 'nn_reg'
model_name_template = f'{model_base_name}_nodes_{{}}'

for node in (8, 16, 32, 64, 128, 256):
    print(f'Processing nodes={node}')
    model_name = model_name_template.format(node)
    model_path = model_dir.joinpath(f'{model_name}.hdf')
    checkpointer = ModelCheckpoint(filepath=str(model_path),
                                   verbose=1, 
                                   save_best_only=True)
    
    if not model_name in hyper_df.loc[:, 'name'].values:
        nn_reg = build_mlp(train_non_dt.shape[1], 
                           'adadelta',
                           rmse_keras_clip,
                           hidden_layers=1,
                           nodes=node,
                           dropout=0.32)
        
        nn_reg.fit(x_train_scaled, 
                   y_train, 
                   validation_data=(x_val_scaled, y_val),
                   batch_size=4096,
                   epochs=10,
                   callbacks=[checkpointer, stopper])
        history = nn_reg.history.history
        pickle_object(model_dir, f'{model_name}', history)
        
        # NOTE: Find the best scores
        nn_reg_val_score = min(nn_reg.history.history['val_loss'])
        index = np.where(np.isclose(history['val_loss'], nn_reg_val_score))[0][0]
        nn_reg_train_score = history['loss'][index]
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, nn_reg_train_score, nn_reg_val_score)
    else:
        nn_reg = load_model(model_path, custom_objects={'rmse_keras_clip': rmse_keras_clip})
        history = unpickle_object(model_dir, f'{model_name}')
        nn_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        nn_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]

In [ ]:
plot_train_val(hyper_df, model_base_name, 'nodes')

The plot is somewhat surprising as the validation error is lower than the training error.
One hypothesis could be that we should have trained for more epochs. However, the model with $256$ nodes ended with early stopping. 

As the score is quite low as compared to the other models we have fitted, we can probe the submit with the last fitted network

In [ ]:
nn_prediction = test_dt.loc[:, ['ID']]
nn_prediction.loc[:, 'item_cnt_month'] = nn_reg.predict(test_non_dt).clip(0, 20)

# Set ID as index
nn_prediction.set_index('ID', inplace=True)

nn_prediction.to_csv(generated_data.joinpath('nn_prediction.csv'))
nn_prediction.loc[:, 'item_cnt_month'].describe()

This gave a submission score of $1.05710$, which is actually not that bad. Although $256$ did not had the lowest score, lets add more layers with $128$ nodes to see if we can improve things a bit. Notice that we increase the number of epochs as more complex networks usually needs more epochs

In [ ]:
model_base_name = 'nn_reg'
model_name_template = f'{model_base_name}_layer_{{}}'

for layer in (2, 3, 4):
    nodes = [256] + [128] * (layer - 1)
    print(f'Processing layers={layer}')
    model_name = model_name_template.format(layer)
    model_path = model_dir.joinpath(f'{model_name}.hdf')
    checkpointer = ModelCheckpoint(filepath=str(model_path),
                                   verbose=1, 
                                   save_best_only=True)
    
    if not model_name in hyper_df.loc[:, 'name'].values:
        nn_reg = build_mlp(train_non_dt.shape[1], 
                           'adadelta',
                           rmse_keras_clip,
                           hidden_layers=1,
                           nodes=nodes,
                           dropout=0.32)
        
        nn_reg.fit(x_train_scaled, 
                   y_train, 
                   validation_data=(x_val_scaled, y_val),
                   batch_size=4096,
                   epochs=100,
                   callbacks=[checkpointer, stopper])
        history = nn_reg.history.history
        pickle_object(model_dir, f'{model_name}', history)
        
        # NOTE: Find the best scores
        nn_reg_val_score = min(nn_reg.history.history['val_loss'])
        index = np.where(np.isclose(history['val_loss'], nn_reg_val_score))[0][0]
        nn_reg_train_score = history['loss'][index]
        hyper_df = add_to_hyper_df(hyper_df, hyper_df_path, model_name, nn_reg_train_score, nn_reg_val_score)
    else:
        nn_reg = load_model(model_path, custom_objects={'rmse_keras_clip': rmse_keras_clip})
        history = unpickle_object(model_dir, f'{model_name}')
        nn_reg_train_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'train_score'].values[0]
        nn_reg_val_score = hyper_df.loc[hyper_df.loc[:, 'name'] == model_name, 'val_score'].values[0]

In [ ]:
plot_train_val(hyper_df, model_base_name, 'layer')

# 6 Ensembling

Let us now investigate the performances of all models

In [ ]:
hyper_df.sort_values('name', axis=0, inplace=True)
hyper_df.reset_index(drop=True, inplace=True)
hyper_df.to_hdf(hyper_df_path, key='hyper_df')
hyper_df

From this we create the best regressors

In [ ]:
best_lin_reg = LinearRegression
best_lasso_reg = Lasso(alpha=)
best_et_reg = ExtraTreesRegressor(n_estimators=75,
                                  max_depth=max_depth,
                                  n_jobs=-1,
                                  random_state=18)
best_xg_reg = XGBRegressor(max_depth=,
                           min_child_weight=,
                           n_jobs=-1,
                           seed=seed)
best_nn_reg = build_mlp(train_non_dt.shape[1], 
                        'adadelta',
                        rmse_keras_clip,
                        hidden_layers=,
                        nodes=,
                        dropout=0.32)

## First level

In the first level we predict with our optimized classifiers, just as we would've done if we were to use only one of the models for the final prediction.

In [ ]:
best_name = 'best_lin_reg'
if not model_dir.joinpath(best_name).is_file():
    best_lin_reg.fit(train_non_dt, target)
    pickle_object(model_dir, best_name, best_lin_reg)
else:
    best_lin_reg = unpickle_object(model_dir, best_name)
    
lin_lvl_1_pred = best_lin_reg.predict(test_non_dt)

In [ ]:
best_name = 'best_lasso_reg'
if not model_dir.joinpath(best_name).is_file():
    best_lasso_reg.fit(train_non_dt, target)
    pickle_object(model_dir, best_name, best_lasso_reg)
else:
    best_lasso_reg = unpickle_object(model_dir, best_name)
    
lasso_lvl_1_pred = best_lasso_reg.predict(test_non_dt)

In [ ]:
best_name = 'best_et_reg'
if not model_dir.joinpath(best_name).is_file():
    best_et_reg.fit(train_dt, target)
    pickle_object(model_dir, best_name, best_et_reg)
else:
    best_et_reg = unpickle_object(model_dir, best_name)
    
et_lvl_1_pred = best_et_reg.predict(test_dt)

In [ ]:
best_name = 'best_xg_reg'
if not model_dir.joinpath(best_name).is_file():
    best_xg_reg.fit(train_dt, target)
    pickle_object(model_dir, best_name, best_xg_reg)
else:
    best_xg_reg = unpickle_object(model_dir, best_name)
    
xg_lvl_1_pred = best_xg_reg.predict(test_dt)

In [ ]:
best_name = 'best_nn_reg'
model_path = model_dir.joinpath(f'{best_name}.hdf')
if not model_dir.joinpath(best_name).is_file():
    checkpointer = ModelCheckpoint(filepath=str(model_path),
                                   verbose=1, 
                                   save_best_only=True)
    best_nn_reg.fit(train_non_dt, target, batch_size=4096, epochs=100, callbacks=[checkpointer, stopper])
else:
    best_nn_reg = = load_model(model_path, custom_objects={'rmse_keras_clip': rmse_keras_clip})
    
nn_lvl_1_pred = best_nn_reg.predict(test_non_dt)

## Second level

Combine the prediction to the new "unseen" level $2$ test set.

First, we will make a test set we can predict the meta model(s) on level $2$ on.

**NOTE**: We will stop after level $2$, so we will only have one meta model on level $2$, but in principle we could have several models on level $2$ to make up the test set of level $3$

In [ ]:
x_test_lvl_2 = np.c_[lin_lvl_1_pred.clip(0, 20), 
                     lasso_lvl_1_pred.clip(0, 20),
                     et_lvl_1_pred.clip(0, 20),
                     xg_lvl_1_pred.clip(0, 20),
                     nn_lvl_1_pred.clip(0, 20)] 

Next, we declare the target for the meta model on level $2$.

As we only have one meta model on level $2$, we will use the $5$ last months to create for the k-fold validation on level $2$. We start by getting the first index of the target

In [ ]:
n_last_month = 5
max_month = train_dt.loc[:, 'date_block_num'].max()
# NOTE: Add one as range has non-inclusive endpoint
month_range = np.array(range(max_month - n_last_month, max_month + 1))
first_index = len(train_dt.loc[train_dt.loc[:, 'date_block_num'].isin(month_range)])

In [ ]:
y_train_lvl_2 = target.iloc[first_index:]

Declare the training set, which we will fill with the k-fold procedure

In [ ]:
x_train_lvl_2 = np.zeros([y_train_level_2.shape[0], x_test_level_2.shape[1]])

In [ ]:
x_train_lvl_2_path = generated_data.joinpath('x_train_lvl_2.hdf')

In [ ]:
index_offset = 0
if not x_train_lvl_2_path.is_file():
    for month in month_range:
        # Find indices
        first_index = len(train_dt.loc[train_dt.loc[:, 'date_block_num'] < month])
        last_index = len(train_dt.loc[train_dt.loc[:, 'date_block_num'] == month])
        
        # Split to train and validate
        x_train_scaled = train_non_dt.iloc[:first_index]
        x_train_unscaled = train_dt.iloc[:first_index]
        y_train = target.iloc[first_index:last_index]
        x_val_scaled = train_non_dt.iloc[first_index:last_index]
        x_val_unscaled = train_dt.iloc[first_index:last_index]
    
        # Make new models (where necessary)
        nn_reg_lvl_2 = build_mlp(x_train_unscaled.shape[1], 
                                 'adadelta',
                                 rmse_keras_clip,
                                 hidden_layers=,
                                 nodes=,
                                 dropout=0.32)    
        
        # Fit
        best_lin_reg.fit(x_train_scaled, y_train)
        best_lasso_reg.fit(x_train_scaled, y_train)
        best_et_reg.fit(x_val_unscaled, y_train)
        best_xg_reg.fit(x_val_unscaled, y_train)
        nn_reg_lvl_2.fit(x_train_scaled, y_train, batch_size=4096, epochs=100, callbacks=[stopper])    
        
        # Predict
        lin_lvl_2_pred = best_lin_reg.predict(x_val_scaled)
        lasso_lvl_2_pred = best_lasso_reg.predict(x_val_scaled)
        et_lvl_2_pred = best_et_reg.predict(x_val_unscaled)
        xg_lvl_2_pred = best_xg_reg.predict(x_val_unscaled)
        nn_lvl_2_pred = nn_reg_lvl_2.predict(x_val_scaled)
    
        # Store the predictions in the training set
        row_inds = np.array(range(index_offset, last_index - first_index + index_offset + 1))
        x_train_lvl_2[row_inds, 0] = lin_lvl_2_pred.clip(0, 20)
        x_train_lvl_2[row_inds, 1] = lasso_lvl_2_pred.clip(0, 20)
        x_train_lvl_2[row_inds, 2] = et_lvl_2_pred.clip(0, 20)
        x_train_lvl_2[row_inds, 3] = xg_lvl_2_pred.clip(0, 20)
        x_train_lvl_2[row_inds, 4] = nn_lvl_2_pred.clip(0, 20)
        
        # Update index_offset
        index_offset = last_index - first_index
    x_train_lvl_2_df = pd.DataFrame(x_train_lvl_2, 
                                    columns=['lin_pred', 'lasso_pred', 'et_pred', 'xg_pred', 'nn_pred'])
    x_train_lvl_2_df.to_hdf(generated_data.joinpath('x_train_lvl_2.hdf'), key='x_train_lvl_2')
else:
    x_train_lvl_2_df = pd.read_hdf(x_train_lvl_2_path, key='x_train_lvl_2')

We now train on the validation months, and predict using the test set for level $2$.

**NOTE**: We are fitting on only the last months, and predicting for the prediction month

In [ ]:
model.fit(x_train_lvl_2_df, y_train_lvl_2)
final_prediction = model.predict(x_test_lvl_2)

In [ ]:
stack_prediction = test_dt.loc[:, ['ID']]
stack_prediction.loc[:, 'item_cnt_month'] = final_prediction.clip(0, 20)

# Set ID as index
stack_prediction.set_index('ID', inplace=True)

stack_prediction.to_csv(generated_data.joinpath('stack_prediction.csv'))
stack_prediction.loc[:, 'item_cnt_month'].describe()